In [ ]:
!pip install neuron

In [ ]:
!nrnivmodl mechanisms

In [ ]:
from neuron import h
from neuron import gui
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

h.load_file('load_L5PC.hoc')

h.topology()

All sections are now defined in HOC, so to access, we have to use `h.<section name>`. Or to make accessing things from python easier you can save the hoc section to a python variable

In [ ]:
soma = h.soma
print(type(soma)) # python neuron section, can use python methods on hoc section
soma.psection()['density_mechs'].keys() # look at channels inserted into soma

Mainen mod files use $pS/\mu m^{2}$, while the `Nav11.mod` and `ichanWT2005.mod` use $S/cm^{2}$ ($1 \; S/cm^2 = 10,000 \; pS/\mu m^2$). 

In [ ]:
print(soma.gbar_na)

total_soma_gna = soma.gbar_na # pS/um^2

# Axon hillock 
hill = h.hill # name of axon hillock section
total_hill_gna = hill.gbar_na

# Add IClamp
iclamp = h.IClamp(soma(0.5))
iclamp.delay = 5
iclamp.dur = 1000
iclamp.amp = 0.2 

t_vec = h.Vector().record(h._ref_t)
soma_v = h.Vector().record(soma(0.5)._ref_v)

Try adding Nav1.1 and setting conductance ratio. A

In [ ]:
soma.insert('nav11')
hill.insert('nav11')

def set_soma_hill_nav11(nav11_ratio,total_soma_gna,total_hill_gna):
    soma.gbar_nav11 = nav11_ratio*total_soma_gna*1e-4 # convert to S/cm2
    soma.gbar_na = (1-nav11_ratio)*total_soma_gna # pS/um^2
    hill.gbar_nav11 = nav11_ratio*total_hill_gna*1e-4 # convert to S/cm2
    hill.gbar_na = (1-nav11_ratio)*total_hill_gna # pS/um^2

In [ ]:
nav11_ratio = 0 # set ratio (0-1) of nav1.1 to nav1.2 density
# Try using nav1.1 in soma/axon hillock
set_soma_hill_nav11(nav11_ratio,total_soma_gna,total_hill_gna)

# Set simulation parameters and run
h.celsius = 37
h.tstop = 1000
h.run()
soma_v_nav11_off = np.array(soma_v) # save somatic voltage recording

# Turn on Nav1.1 exclusively
nav11_ratio = 1
set_soma_hill_nav11(nav11_ratio,total_soma_gna,total_hill_gna)
h.run()
soma_v_nav11_on = np.array(soma_v) # save somatic voltage recording

# Plot 

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(t_vec,soma_v_nav11_off,label='nav11 off')
ax.plot(t_vec,soma_v_nav11_on,label='nav11 on')
ax.set_xlabel('time (ms)')
ax.set_ylabel('Vm (mV)');
ax.set_ylim(-80,50)
ax.legend();
# Reset to normal
set_soma_hill_nav11(0,total_soma_gna,total_hill_gna)
